# Retrieve Data from Onemap API

### Loading Libraries and API Configurations

Note that Onemap Token only last for 3 days, and have to be renewed if we want to retrieve data from OneMap again

In [1]:
# General Data  Tools
import pandas as pd
import numpy as np
import os
import json

# Geospatial
import geopandas as gpd
import geojson
from shapely.geometry import Polygon, MultiPolygon, shape

# Utilities
from utility import retrieve_from_onemap, convert_geojson_to_geometry, export_df_to_shapefile, retrieve_onemap_population_data

# Load API Token
from dotenv import load_dotenv
load_dotenv()
onemap_token = os.getenv('onemap_token')

import warnings
warnings.filterwarnings("ignore")

## Geocoding (Testing)

In [4]:
df_test = retrieve_from_onemap(onemap_token, "/commonapi/search", payload={"searchVal": "9C Boon Tiong Rd", "returnGeom": "Y", "getAddrDetails": "Y"})
df_test.head()

,found,totalNumPages,pageNum,results
0,1,1,1,"{'SEARCHVAL': 'TIONG BAHRU VIEW', 'BLK_NO': '9..."


In [5]:
df_test.loc[0, "results"]

{'SEARCHVAL': 'TIONG BAHRU VIEW',
 'BLK_NO': '9C',
 'ROAD_NAME': 'BOON TIONG ROAD',
 'BUILDING': 'TIONG BAHRU VIEW',
 'ADDRESS': '9C BOON TIONG ROAD TIONG BAHRU VIEW SINGAPORE 164009',
 'POSTAL': '164009',
 'X': '27533.0640224759',
 'Y': '29918.2532074675',
 'LATITUDE': '1.28684455991278',
 'LONGITUDE': '103.829122970966',
 'LONGTITUDE': '103.829122970966'}

## Planning Area Query 

Earliest Valid Year: 2014

Skip this section if Planning Area data is already in Data folder

In [2]:
# Raw Data Containing Geojson
df_planning_area = retrieve_from_onemap(onemap_token, "privateapi/popapi/getAllPlanningarea", payload={"year": "2014"})
df_planning_area.head()

,pln_area_n,geojson
0,SIMPANG,"{""type"":""MultiPolygon"",""coordinates"":[[[[103.8..."
1,SOUTHERN ISLANDS,"{""type"":""MultiPolygon"",""coordinates"":[[[[103.8..."
2,SUNGEI KADUT,"{""type"":""MultiPolygon"",""coordinates"":[[[[103.7..."
3,TUAS,"{""type"":""MultiPolygon"",""coordinates"":[[[[103.6..."
4,WESTERN ISLANDS,"{""type"":""MultiPolygon"",""coordinates"":[[[[103.7..."


In [3]:
df_planning_area = convert_geojson_to_geometry(df_planning_area)
df_planning_area.head()

,pln_area_n,geometry
0,SIMPANG,"(POLYGON ((103.859314 1.439057, 103.859556 1.4..."
1,SOUTHERN ISLANDS,"(POLYGON ((103.835249 1.213345, 103.835104 1.2..."
2,SUNGEI KADUT,"(POLYGON ((103.766276 1.441204, 103.766367 1.4..."
3,TUAS,"(POLYGON ((103.648439 1.239805, 103.64855 1.23..."
4,WESTERN ISLANDS,"(POLYGON ((103.741276 1.160871, 103.741342 1.1..."


Unknown Planning Region names ```OTHERS``` that has no geometry information is dropped

In [7]:
df_planning_area[df_planning_area["geometry"].isna()]

,pln_area_n,geometry
18,OTHERS,None


In [8]:
df_planning_area.dropna(subset=["geometry"], axis=0, inplace=True)
print(type(df_planning_area))
print(type(df_planning_area["geometry"]))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


In [9]:
gdf_planning_area = export_df_to_shapefile(df_planning_area, "onemap_planning_area")
gdf_planning_area.head()

,pln_area_n,geometry
0,SIMPANG,"MULTIPOLYGON (((103.85931 1.43906, 103.85956 1..."
1,SOUTHERN ISLANDS,"MULTIPOLYGON (((103.83525 1.21334, 103.83510 1..."
2,SUNGEI KADUT,"MULTIPOLYGON (((103.76628 1.44120, 103.76637 1..."
3,TUAS,"MULTIPOLYGON (((103.64844 1.23981, 103.64855 1..."
4,WESTERN ISLANDS,"MULTIPOLYGON (((103.74128 1.16087, 103.74134 1..."


In [6]:
print(type(gdf_planning_area))
print(type(gdf_planning_area["geometry"]))

<class 'geopandas.geodataframe.GeoDataFrame'>
<class 'geopandas.geoseries.GeoSeries'>


## Population Query

Earliest Valid Year: 2020

### Retrieve list of planning areas 
From Data Folder

In [2]:
gdf_planning_area = gpd.read_file("../data/onemap_planning_area/onemap_planning_area.shp")
lst_of_planning_area = list(gdf_planning_area.loc[:,"pln_area_n"])

### Transportation Related
- getModeOfTransportSchool
- getModeOfTransportWork

In [3]:
df_mode_of_transport_sch = retrieve_onemap_population_data(lst_of_area_names=lst_of_planning_area, token=onemap_token, path="privateapi/popapi/getModeOfTransportSchool", year=2020)
df_mode_of_transport_sch.sample(5)

,planning_area,bus,mrt,mrt_bus,mrt_car,mrt_other,taxi,car,pvt_chartered_bus,lorry_pickup,motorcycle_scooter,others,no_transport_required,other_combi_mrt_or_bus,mrt_lrt_only,mrt_lrt_and_bus,other_combi_mrt_lrt_or_bus,taxi_pvt_hire_car_only,pvt_chartered_bus_van,year
3,TUAS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2020
45,WOODLANDS,9383,0,0,0,0,0,3428,0,99,255,305,14819,0,4143,14689,778,309,1457,2020
7,NEWTON,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2020
46,YISHUN,7903,0,0,0,0,0,4109,0,47,244,756,10444,0,2837,9630,466,200,1519,2020
37,HOUGANG,9819,0,0,0,0,0,4888,0,51,153,336,7841,0,1777,9269,538,292,1612,2020


In [7]:
df_mode_of_transport_work = retrieve_onemap_population_data(lst_of_area_names=lst_of_planning_area, token=onemap_token, path="privateapi/popapi/getModeOfTransportWork", year=2020)
df_mode_of_transport_work.sample(5)

,planning_area,bus,mrt,mrt_bus,mrt_car,mrt_other,taxi,car,pvt_chartered_bus,lorry_pickup,motorcycle_scooter,others,no_transport_required,other_combi_mrt_or_bus,mrt_lrt_only,mrt_lrt_and_bus,other_combi_mrt_lrt_or_bus,taxi_pvt_hire_car_only,pvt_chartered_bus_van,year
16,TENGAH,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2020
37,HOUGANG,22743,0,0,0,0,0,26845,0,2200,3457,1867,12224,0,12909,35473,2608,3293,2116,2020
27,QUEENSTOWN,8856,0,0,0,0,0,9251,0,353,841,428,5015,0,8542,11023,944,1766,606,2020
42,TAMPINES,25058,0,0,0,0,0,26251,0,1351,6762,3052,13937,0,20843,34708,3591,4357,2972,2020
30,ANG MO KIO,14928,0,0,0,0,0,16343,0,1454,2095,976,8822,0,10689,24760,1651,2204,1567,2020


In [28]:
df_mode_of_transport_sch.to_csv("../data/onemap_mode_of_transport_school.csv", index=False, encoding="utf-8")
df_mode_of_transport_work.to_csv("../data/onemap_mode_of_transport_work.csv", index=False, encoding="utf-8")

### Population Related

- getPopulationAgeGroup

In [6]:
df_pop_age_grp = retrieve_onemap_population_data(lst_of_area_names=lst_of_planning_area, token=onemap_token, path="privateapi/popapi/getPopulationAgeGroup", year=2020)
df_pop_age_grp.sample(5)

,planning_area,age_0_4,age_5_9,age_10_14,age_15_19,age_20_24,age_25_29,age_30_34,age_35_39,age_40_44,...,age_55_59,age_60_64,age_65_69,age_70_74,age_75_79,age_80_84,age_85_over,total,gender,year
159,ROCHOR,210,200,230,240,310,340,460,490,540,...,520,530,440,350,220,150,240,6530,Male,2020
85,SELETAR,10,10,10,10,10,0,0,10,20,...,20,10,10,10,0,0,0,160,Female,2020
86,SELETAR,0,10,10,10,20,0,10,0,10,...,10,10,10,0,0,0,0,140,Male,2020
1,SIMPANG,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Total,2020
14,WESTERN ISLANDS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Male,2020


In [5]:
df_pop_age_grp["planning_area"] = df_pop_age_grp["planning_area"].apply(lambda x: x.upper())
df_pop_age_grp.to_csv("../data/onemap_population_age_group.csv", index=False, encoding="utf-8")